# 초음파 광물 예측

### 데이터

In [9]:
import numpy as np
import tensorflow as tf

In [10]:
# 실행할 때마다 같은 결과를 출력하기 위한 seed 값 설정
seed = 2021
np.random.seed(seed)
tf.random.set_seed(seed)

In [11]:
from google.colab import files
uploaded_file = files.upload()
filename = list(uploaded_file.keys())[0]

Saving sonar.csv to sonar (2).csv


In [13]:
import pandas as pd
# df = pd.read_csv('dataset/sonar.csv', header=None)
df = pd.read_csv(filename, header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,0.1609,0.1582,0.2238,0.0645,0.0660,0.2273,0.3100,0.2999,0.5078,0.4797,0.5783,0.5071,0.4328,0.5550,0.6711,0.6415,0.7104,0.8080,0.6791,0.3857,0.1307,0.2604,0.5121,0.7547,0.8537,0.8507,0.6692,0.6097,0.4943,0.2744,0.0510,0.2834,0.2825,0.4256,0.2641,0.1386,0.1051,0.1343,0.0383,0.0324,0.0232,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,R
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,0.4918,0.6552,0.6919,0.7797,0.7464,0.9444,1.0000,0.8874,0.8024,0.7818,0.5212,0.4052,0.3957,0.3914,0.3250,0.3200,0.3271,0.2767,0.4423,0.2028,0.3788,0.2947,0.1984,0.2341,0.1306,0.4182,0.3835,0.1057,0.1840,0.1970,0.1674,0.0583,0.1401,0.1628,0.0621,0.0203,0.0530,0.0742,0.0409,0.0061,0.0125,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,R
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,0.6333,0.7060,0.5544,0.5320,0.6479,0.6931,0.6759,0.7551,0.8929,0.8619,0.7974,0.6737,0.4293,0.3648,0.5331,0.2413,0.5070,0.8533,0.6036,0.8514,0.8512,0.5045,0.1862,0.2709,0.4232,0.3043,0.6116,0.6756,0.5375,0.4719,0.4647,0.2587,0.2129,0.2222,0.2111,0.0176,0.1348,0.0744,0.0130,0.0106,0.0033,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,R
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,0.0881,0.1992,0.0184,0.2261,0.1729,0.2131,0.0693,0.2281,0.4060,0.3973,0.2741,0.3690,0.5556,0.4846,0.3140,0.5334,0.5256,0.2520,0.2090,0.3559,0.6260,0.7340,0.6120,0.3497,0.3953,0.3012,0.5408,0.8814,0.9857,0.9167,0.6121,0.5006,0.3210,0.3202,0.4295,0.3654,0.2655,0.1576,0.0681,0.0294,0.0241,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,R
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,0.4152,0.3952,0.4256,0.4135,0.4528,0.5326,0.7306,0.6193,0.2032,0.4636,0.4148,0.4292,0.5730,0.5399,0.3161,0.2285,0.6995,1.0000,0.7262,0.4724,0.5103,0.5459,0.2881,0.0981,0.1951,0.4181,0.4604,0.3217,0.2828,0.2430,0.1979,0.2444,0.1847,0.0841,0.0692,0.0528,0.0357,0.0085,0.0230,0.0046,0.0156,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,R


In [14]:
df[60].unique()

array(['R', 'M'], dtype=object)

In [15]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_encoded = le.fit_transform(df[60].values)

In [16]:
X = df.iloc[:,:-1].values
X.shape

(208, 60)

### 과적합 발생

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Input
model = Sequential([
    Dense(24, input_dim=60, activation='relu'),
    Dense(8, activation='relu'),
    Dense(1, activation='sigmoid')
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 24)                1464      
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 200       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 9         
Total params: 1,673
Trainable params: 1,673
Non-trainable params: 0
_________________________________________________________________


In [18]:
model.compile(
                loss='binary_crossentropy', 
                optimizer='adam', 
                metrics=['accuracy']
            )

In [19]:
history = model.fit(X,y_encoded, epochs=200, batch_size=10,verbose=0)

In [20]:
acc = model.evaluate(X, y_encoded, verbose=2)
print(f"Accuracy: {acc[1]:.4f}")

7/7 - 0s - loss: 0.0313 - accuracy: 1.0000
Accuracy: 1.0000


## test,train 으로 나누기

In [21]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_tran,y_test = train_test_split(
    X,y_encoded,stratify=y_encoded,random_state=seed
)

In [22]:
X_train.shape,X_test.shape

((156, 60), (52, 60))

In [23]:
model2 = Sequential([
    Dense(24, input_dim=60, activation='relu'),
    Dense(8, activation='relu'),
    Dense(1, activation='sigmoid')
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 24)                1464      
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 200       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 9         
Total params: 1,673
Trainable params: 1,673
Non-trainable params: 0
_________________________________________________________________


In [24]:
model2.compile(
                loss='binary_crossentropy', 
                optimizer='adam', 
                metrics=['accuracy']
            )

In [25]:
import os
MODEL_DIR = './model/'
if not os.path.exists(MODEL_DIR):
    os.mkdir(MODEL_DIR)

In [26]:
!ls

 model	 sample_data  'sonar (1).csv'  'sonar (2).csv'	 sonar.csv


In [27]:
# 모델 저장 조건
modelpath = MODEL_DIR + "best{epoch:03d}-{val_loss:.4f}.hdf5"

In [28]:
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', verbose=1, save_best_only=True)
early_stopping = EarlyStopping(patience=30)

In [29]:
history = model2.fit(X_train,y_tran,validation_split=0.2, epochs=200, batch_size=10,verbose=0,callbacks=[checkpointer,early_stopping])


Epoch 00001: val_loss improved from inf to 0.65971, saving model to ./model/best001-0.6597.hdf5

Epoch 00002: val_loss improved from 0.65971 to 0.64908, saving model to ./model/best002-0.6491.hdf5

Epoch 00003: val_loss improved from 0.64908 to 0.63863, saving model to ./model/best003-0.6386.hdf5

Epoch 00004: val_loss improved from 0.63863 to 0.63010, saving model to ./model/best004-0.6301.hdf5

Epoch 00005: val_loss improved from 0.63010 to 0.61821, saving model to ./model/best005-0.6182.hdf5

Epoch 00006: val_loss improved from 0.61821 to 0.60977, saving model to ./model/best006-0.6098.hdf5

Epoch 00007: val_loss improved from 0.60977 to 0.60556, saving model to ./model/best007-0.6056.hdf5

Epoch 00008: val_loss improved from 0.60556 to 0.59231, saving model to ./model/best008-0.5923.hdf5

Epoch 00009: val_loss improved from 0.59231 to 0.58263, saving model to ./model/best009-0.5826.hdf5

Epoch 00010: val_loss improved from 0.58263 to 0.58108, saving model to ./model/best010-0.5811

In [31]:
from tensorflow.keras.models import load_model
best_model = load_model('./model/best072-0.3726.hdf5')
acc = best_model.evaluate(X_test, y_test)
print(f"Accuracy: {acc[1]:.4f}")

2/2 [==============================] - 0s 6ms/step - loss: 0.4469 - accuracy: 0.8077
Accuracy: 0.8077


## 모델 저장하기

In [ ]:
model2.save('my_model2.h5')

In [ ]:
!ls

dataset  my_model2.h5  sample_data


In [ ]:
# 모델 pc로 다운로드
# files.download('my_model2.h5')

In [ ]:
from tensorflow.keras.models import load_model
my_model = load_model('my_model2.h5')
acc = my_model.evaluate(X_test,y_test)
print(f"Accuracy: {acc[1]:.4f}")

2/2 [==============================] - 0s 6ms/step - loss: 0.5265 - accuracy: 0.8269
Accuracy: 0.8269
